In [46]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import os


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-xxx"  

In [48]:
system_prompt = (
    '''You are a performance feedback assistant for managers. 
Your job is to rewrite manager notes into professional, constructive, and encouraging feedback for employees. 
Always start with positive points, clearly suggest improvements, and end with motivating encouragement. 
Keep a professional, supportive, and concise tone. 
Refine your answers according to manager feedback and any tips provided.

Employee Name: {employee_name}
Manager's Notes: {performance_notes}
AI Tips: {external_tips}

Generate a professional feedback paragraph that:
- Starts with positive observations
- Uses ONLY information provided in the manager's notes - DO NOT add fictional details, projects, or achievements
- Addresses areas for improvement constructively based on what's mentioned
- Ends with encouragement and next steps
- Uses professional, supportive tone
- Is 3-4 sentences long
- Avoids generic phrases
- Incorporates the AI tips provided
- NEVER assume or invent specific projects, campaigns, or achievements not mentioned

Return ONLY the feedback text, no additional formatting or explanations.
'''
)

In [49]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5,   
    streaming=False
)

In [50]:
llm.predict(system_prompt)

external_tips = "Focus on teamwork and leadership examples. Keep tone encouraging but concise."

In [51]:
memory = ConversationBufferMemory(return_messages=True)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  
)

In [55]:
current_feedback = None

while True:
    if current_feedback is None:
        employee_name = input("Employee Name: ")
        if employee_name.lower() in ["exit", "quit"]:
            break

        manager_feedback = input("Manager Feedback: ")
        if manager_feedback.lower() in ["exit", "quit"]:
            break

        prompt = f"{system_prompt}\nEmployee: {employee_name}\nNotes: {manager_feedback}\nTips: {external_tips}"

        print("Employee name:", employee_name)
        print("Manager feedback:", manager_feedback)
        print("AI tips:", external_tips)
        print("Full prompt:\n", prompt)

        current_feedback = conversation.predict(input=prompt)
        print("\nAssistant:", current_feedback, "\n")

    else:
        refine = input("Do you want to refine this feedback? (yes/no): ")
        if refine.lower() not in ["yes", "y"]:
            print("Final feedback accepted.")
            break

        edit_feedback = input("Manager Edits / Refinements: ")
        if edit_feedback.lower() in ["exit", "quit"]:
            print("Exiting assistant. Goodbye!")
            break

        prompt = (
            f"Previous feedback:\n{current_feedback}\n\n"
            f"New instructions / edits:\n{edit_feedback}\n\n"
            f"Tips: {external_tips}\n"
            "Refine the previous feedback accordingly, keep it professional and encouraging, "
            "and do NOT invent details."
        )

        current_feedback = conversation.predict(input=prompt)
        print("\nAssistant:", current_feedback, "\n")


Employee name: jOHN
Manager feedback: John is hardowrking
AI tips: Focus on teamwork and leadership examples. Keep tone encouraging but concise.
Full prompt:
 You are a performance feedback assistant for managers. 
Your job is to rewrite manager notes into professional, constructive, and encouraging feedback for employees. 
Always start with positive points, clearly suggest improvements, and end with motivating encouragement. 
Keep a professional, supportive, and concise tone. 
Refine your answers according to manager feedback and any tips provided.

Employee Name: {employee_name}
Manager's Notes: {performance_notes}
AI Tips: {external_tips}

Generate a professional feedback paragraph that:
- Starts with positive observations
- Uses ONLY information provided in the manager's notes - DO NOT add fictional details, projects, or achievements
- Addresses areas for improvement constructively based on what's mentioned
- Ends with encouragement and next steps
- Uses professional, supportive to